In [2]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate
import os

from spectrum import FitsSpectrum
from common import list_files, list_directories, tqdm

In [ ]:
dib_centers_all = [float(dib_file.replace('.ascii', '')) for dib_file in os.listdir('data/dibs')]
dib_centers_subset = [5780,5797,6196,6379,6613,7224]

def load_target(target_dir: str) -> list[FitsSpectrum]:
    return [FitsSpectrum(datafile) for band_dir in list_directories(target_dir) for datafile in list_files(band_dir)]

def fit_gaussian_for_targets(targets: list | str, dib_centra_list: list, export = False, create_figure = False):
    headers = ['Observation date', 'Center [Å]', 'Width', 'Amplitude', 'Skew', 'RMSE', 'FWHM [Å]', 'EW [Å]']
    table_data = []

    if type(targets) == str:
        targets = [targets]

    targets_tqdm = tqdm(targets)
    for target in targets_tqdm:
        export_data = ['# ' + '\t'.join(headers)]
        subspectra = load_target(f'data/fits/{target}')
        
        for subspectrum in subspectra:
            wvl_max, wvl_min = np.max(subspectrum.wavelength), np.min(subspectrum.wavelength)

            subspectrum.remove_outliers()
            subspectrum.normalize(max_degree=10)
            subspectrum.correct_shift()

            for center in dib_centra_list:
                targets_tqdm.set_description(f'{target} | λ={wvl_min:.2f}-{wvl_max:.2f} [{center:.2f}]')

                # The DIB does not lie in this subspectrum
                if not wvl_min < center < wvl_max:
                    continue

                ax = None
                if create_figure:
                    _, ax = plt.subplots(1, 1, figsize=(15, 9))

                result = subspectrum.select_dib(center, ax=ax)

                if result is None:
                    continue

                params, rmse, fwhm, ew = result
                centers, widths, amplitudes, skews = np.column_stack(params)

                table_data.append([subspectrum.target, subspectrum.format_obs_date(), *np.column_stack(params), rmse, fwhm, ew])

                if export:
                    export_data.append('\t'.join([
                        subspectrum.obs_date.isoformat(),
                        np.array2string(centers, separator=','),
                        np.array2string(amplitudes, separator=','),
                        np.array2string(widths, separator=','),
                        np.array2string(skews, separator=','),
                        str(rmse), str(fwhm), str(ew)
                    ]))

        if export:
            export_file = open(f'out/{target}_gaussians.csv', 'w+')
            export_file.write('\n'.join(export_data))
            export_file.close()

    return tabulate(table_data, ['Target', *headers], tablefmt='html')


In [ ]:
fit_gaussian_for_targets('HD185859', dib_centers_subset, create_figure=True, export=True)

In [7]:
_ = fit_gaussian_for_targets(['HD147889', 'HD158926', 'HD170740', 'HD185859'], dib_centers_all, export=True)


HD185859 | λ=8522.92-10429.48 [6194.70]: 100%|██████████| 4/4 [26:06<00:00, 391.65s/it]


In [ ]:
fit_gaussian_for_targets(os.listdir('data/fits'), filename='gaussian_fits_all.csv')